In [ ]:
from sqlalchemy import create_engine
import pandas as pd
from passwords import password as pw

In [ ]:

engine = create_engine(f'postgresql+psycopg2://{pw}@localhost:5432/HomeWork9')
connection = engine.connect()